# #6 Nonogram - Formulation
*Authors: Éder Pinheiro, Aster Santana*  
*August, 2021*

This is the MIP formulation of the puzzle. Statement and solution implementation of all puzzles 
are available from the main page of the [Fun Puzzles](https://mip-master.github.io/puzzles/) project, 
which is maintained by [Mip Master](https://mipmaster.org/).

## Formulation

The purpose of the game is to paint some of the cells so that in every row and every collumn the painted cells form distinct strings with the lengths presented in correct order by the numbers on the left of the rows and on top of the columns.

<img src="../../figures/6_nonogram.png" alt="drawing" width="450"/>

### Input Data

- rows:  
    ```python
    I = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
    ```
- columns:  
    ```python
    J = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
    ```
- Row strings length:  
    ```python
    RS = {
        1: [4, 1, 2], 
        2: [3, 1], 
        3: [1, 3, 2], 
        4: [1, 1, 1, 2], 
        5: [1, 1, 1, 1],
        6: [1, 1, 1, 1], 
        7: [1, 1, 1, 4], 
        8: [1, 3], 
        9: [1, 4, 2], 
        10: [2, 2]
        }
    ```
- Column strings length:  
    ```python
    CS = {
        1: [1, 1, 3], 
        2: [1, 1, 1], 
        3: [1, 1, 1], 
        4: [1, 2, 1], 
        5: [3, 3], 
        6: [3, 2, 3], 
        7: [3, 4], 
        8: [1, 1], 
        9: [1, 2, 1, 2], 
        10: [6, 2]
        }
    ```

Notice that $RS[1][0]$ returns $4$, which is the length of the first string of the first row. As another example, $CS[5][1]$ returns $3$, which is the length of the third string of the fith column. We will use this notation in the formulation below.

### Decision Variables
The obvious decisions we need to make is which cell get color to form strings. For that, we define the following set of decision variables:
- $x_{ij}$ equals $1$ if cell $(i, j)$ gets color, $0$ otherwise.
With these varaibles only, it's hard to model the number of strings in each row and column, its length and position.

So we define two more set of variables:
- $y_{ijk}$ equals $1$ is the k-th string of row $i$ begins in column $j$, $0$ otherwise.
- $z_{ijk}$ equals $1$ is the k-th string of column $j$ begins in row $i$, $0$ otherwise.


### Constraints
Let's start with the constraints that garantee that every string begins once. This may be obvious but must be modeled.

- *Each row string begins in exactly one column*:
$$\sum_j y_{ijk} = 1 \quad \forall i \in I, k \in RS[i].$$

- *Each column string begins in exactly one row*:
$$\sum_i z_{ijk} = 1 \quad \forall j \in J, k \in CS[i].$$

Next, we must ensure that every strings has the length it's supposed to have. For example, we know that the first string of the first row must have length $4$. So if it starts at column $2$, then cells $(1, 2), (1, 3), (1, 4)$, and $(1, 5)$ must be filled. In terms of decision variables, this means that, if $y_{121}=1$, then $x_{12}, x_{13}, x_{14}, x_{15} = 1$. This implication connects the $y$ variables with the $x$ variables and can be formulated as follows:
$$
\begin{align*}
y_{121} &\leq x_{12}\\
y_{121} &\leq x_{13}\\
y_{121} &\leq x_{14}\\
y_{121} &\leq x_{15}.
\end{align*}
$$
Alternatively, this could be formulated as a single constraint:
$$4y_{121} \leq x_{12} + x_{13} + x_{14} + x_{15}.$$
Although this second option is more compact and looks nicer, this is usually less computationally efficient because it leads to a weaker formulation, i.e., it's LP relaxation contains the LP relaxation of the first desaggragated formulation.

Next, we write this constrain in its generic form for rows and columns.
- *Row strings length*:
$$y_{ijk} \leq x_{i,j+t}, \quad \forall  i, j, k, \forall t < RS[i][k].$$

- *Column strings length*:
$$z_{ijk} \leq x_{i+t,j}, \quad \forall  i, j, k, \forall t < CS[j][k].$$

We are not done yet. Suppose the first string of the first row begins at column $2$, like in our example above. Now, what happens if the second string begins at column $6$? In this case, the first and second string would become a single string that would be $5$ cells long. There is nothing in the formulatin so far that would prevent that. So we need to ensure that will exist at least one empty cell between any two subsequent strings. Here is one way we can acouplish this in our example. If the first string of the first row begins at column $2$, i.e., $y_{121} = 1$, then we want $y_{1j2}=0$ for all $j\leq 2+4+1=6$ (in the sum, $2$ is the starting column, $4$ is the length of the string, and $1$ is an empty cell to keep the first string disconnected from the second string). Notice that this also ensure that the second string will only come after the first string--not before, nor overlapping, nor connect, but after. This implication can be formulated as follows:
$$y_{1j2} \leq 1 - y_{121} \quad \forall j \in \{0, 1, \cdots, 6\}.$$

- *Row strings disjunction and precedence*:
$$y_{i,j',k+1} \leq 1 - y_{ijk} \quad \forall i, j, k, \forall j'\leq j+RS[i][k]+1.$$

- *Column strings disjunction and precedence*:
$$z_{i',j,k+1} \leq 1-z_{ijk} \quad \forall i, j, k, \forall i'\leq i+CS[j][k]+1.$$

### Objective Function
We set objective to minimize the sum of the $x$ variables. This will garantee that only cells composing a defined string will get filled.
$$\min \sum_{i, j} x_{ij}.$$

### Final Formulation
$$
\begin{align}
& \min & \sum_{i, j} x_{ij}\\
& \text{s.t.}& \sum_j y_{ijk} &= 1 \quad \forall i, k \in RS[i]\\
&& \sum_i z_{ijk} &= 1 \quad \forall j, k \in CS[i]\\
&& y_{ijk} &\leq x_{i,j+t}, \quad \forall  i, j, k, \; \forall t < RS[i][k]\\
&& z_{ijk} &\leq x_{i+t,j}, \quad \forall  i, j, k, \; \forall t < CS[j][k]\\
&& y_{ijk} &\leq 1-y_{i,j',k+1} \quad \forall i, j, k, \; \forall j'\leq j+RS[i][k]+1\\
&& z_{ijk} &\leq 1-z_{i',j,k+1} \quad \forall i, j, k, \; \forall i'\leq i+CS[j][k]+1\\
&& x_{ij}, y_{ijk}, z_{ijk} &\in \{0, 1\}, \quad \forall i, j, k.
\end{align}
$$

### Nanogram solution

<img src="nonogram_solution.png" alt="drawing" width="400"/>

In [1]:
"""
Solution to the Nonogram puzzle.

This version uses SCIP as a solver.

Created by Aster Santana and Éder Pinheiro (Aug, 2021), MipMaster.org.
"""

from pyscipopt import Model, quicksum
from collections import OrderedDict

# region Input Data
# row strings
# using ordered dictionaries because the order of strings in each row/column matters in the implementation
RS = {
  1: OrderedDict({1: 4, 2: 1, 3: 2}),
  2: OrderedDict({1: 3, 2: 1}),
  3: OrderedDict({1: 1, 2: 3, 3: 2}),
  4: OrderedDict({1: 1, 2: 1, 3: 1, 4: 2}),
  5: OrderedDict({1: 1, 2: 1, 3: 1, 4: 1}),
  6: OrderedDict({1: 1, 2: 1, 3: 1, 4: 1}),
  7: OrderedDict({1: 1, 2: 1, 3: 1, 4: 4}),
  8: OrderedDict({1: 1, 2: 3}),
  9: OrderedDict({1: 1, 2: 4, 3: 2}),
  10: OrderedDict({1: 2, 2: 2})
  }
# column strings
CS = {
  1: OrderedDict({1: 1, 2: 1, 3: 3}),
  2: OrderedDict({1: 1, 2: 1, 3: 1}),
  3: OrderedDict({1: 1, 2: 1, 3: 1}),
  4: OrderedDict({1: 1, 2: 2, 3: 1}),
  5: OrderedDict({1: 3, 2: 3}),
  6: OrderedDict({1: 3, 2: 2, 3: 3}),
  7: OrderedDict({1: 3, 2: 4}),
  8: OrderedDict({1: 1, 2: 1}),
  9: OrderedDict({1: 1, 2: 2, 3: 1, 4: 2}),
  10: OrderedDict({1: 6, 2: 2})
  }
# rows
I = list(RS.keys())
# columns
J = list(CS.keys())
# keys for decision variables
x_keys = [(i, j) for i in I for j in J]
y_keys = [(i, j, k) for i in I for j in J for k in RS[i].keys()]
z_keys = [(i, j, k) for i in I for j in J for k in CS[j].keys()]
# endregion

# region Define the model
mdl = Model('nonogram')

# add variables
x, y, z = dict(), dict(), dict()
for key in x_keys:
    x[key] = mdl.addVar(vtype='B', name=f'x_{key}')
for key in y_keys:
    y[key] = mdl.addVar(vtype='B', name=f'y_{key}')
for key in z_keys:
    z[key] = mdl.addVar(vtype='B', name=f'z_{key}')

# add constraints
# each row string begins in exactly one column
for i in I:
    for k in RS[i].keys():
        mdl.addCons(quicksum(y[i, j, k] for j in J) == 1, name=f'str_row{i}_{k}')
# each column string begins in exactly one row
for j in J:
    for k in CS[j].keys():
        mdl.addCons(quicksum(z[i, j, k] for i in I) == 1, name=f'str_col{j}_{k}')
# row strings length
for i in I:
    for j in J:
        for k in RS[i].keys():
            for t in range(RS[i][k]):
                mdl.addCons(y[i, j, k] <= x.get((i, j+t), 0), name=f'row_len_{i}_{j}_{k}_{t}')
# column strings length
for i in I:
    for j in J:
        for k in CS[j].keys():
            for t in range(CS[j][k]):
                mdl.addCons(z[i, j, k] <= x.get((i+t, j), 0), name=f'col_len_{i}_{j}_{k}_{t}')
# row strings disjunction and precedence
for i in I:
    for j in J:
        for k in RS[i].keys():
            for jp in range(1, j + RS[i][k]+1):  # the +1 ensures disjunction, i.e., an empty cell between strings
                mdl.addCons(y.get((i, jp, k+1), 0) <= 1 - y[i, j, k], name=f'row_pre_{i}_{j}_{k}_{jp}')
# # column strings disjunction and precedence
for i in I:
    for j in J:
        for k in CS[j].keys():
            for ip in range(1, i + CS[j][k]+1):  # the +1 ensures disjunction, i.e., an empty cell between strings
                mdl.addCons(z.get((ip, j, k+1), 0) <= 1 - z[i, j, k], name=f'col_pre_{i}_{j}_{k}_{ip}')
# set the objective function
mdl.setObjective(quicksum(x[key] for key in x_keys))
# endregion

# region Optimize and retrieve the solution
mdl.optimize()

# retrieve and print out the solution
for i in I:
    row = [int(mdl.getVal(x[i, j])) for j in J]
    print(row)
# endregion


ModuleNotFoundError: No module named 'pyscipopt'

In [2]:
pip install pyscipopt

  Using cached PySCIPOpt-3.2.2.tar.gz (633 kB)
  ERROR: Command errored out with exit status 1:
   command: /Users/epinheiro/workspace/shipping-vrp-lib/shipping-vrp-lib-bench/src/main/resources/scripts/.scripts_venv/bin/python -u -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'/private/var/folders/72/rqs2pwxs7gz9_ldhbvj2pgc4wb2z4_/T/pip-install-ibrzkou5/pyscipopt_afcb9fc3642148e988aff9f17c537b75/setup.py'"'"'; __file__='"'"'/private/var/folders/72/rqs2pwxs7gz9_ldhbvj2pgc4wb2z4_/T/pip-install-ibrzkou5/pyscipopt_afcb9fc3642148e988aff9f17c537b75/setup.py'"'"';f = getattr(tokenize, '"'"'open'"'"', open)(__file__) if os.path.exists(__file__) else io.StringIO('"'"'from setuptools import setup; setup()'"'"');code = f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' bdist_wheel -d /private/var/folders/72/rqs2pwxs7gz9_ldhbvj2pgc4wb2z4_/T/pip-wheel-rwb9jst1
       cwd: /private/var/folders/72/rqs2pwxs7gz9_ldhbvj2pgc4wb2z4_/T/p